In [ ]:
%load_ext autoreload
%autoreload 2

import crud as crud
import pandas as pd
import matplotlib

from sklearn.model_selection import train_test_split

from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
MIN_PLAYCOUNT = 5
MAX_PLAYCOUNT = 150
MIN_RATING = 30
SEED = 2020
TEST_RATIO = 0.33

In [ ]:
train_triplets = pd.read_csv("./triplets_train_sample.csv", sep=",")
test_triplets = pd.read_csv("./triplets_test_sample.csv", sep=",")
triplets = pd.concat([train_triplets, test_triplets]) 

triplets['user_id_idx'] = triplets.groupby('user_id').ngroup()
triplets['song_id_idx'] = triplets.groupby('song_id').ngroup()

user_mapping = triplets[['user_id_idx', 'user_id']].drop_duplicates()
song_mapping = triplets[['song_id_idx', 'song_id']].drop_duplicates()

train_triplets = train_triplets.merge(user_mapping, on="user_id").merge(song_mapping, on='song_id')
test_triplets = test_triplets.merge(user_mapping, on="user_id").merge(song_mapping, on='song_id')

In [ ]:
triplets.head()

,user_id,song_id,play_count,user_id_idx,song_id_idx
0,41536eb0847ad7aa746a29d81367be6bc9f676fd,SOVGSEL12A8C141591,13,10612,18158
1,350e7d679291614f8678a77c17c5fa790721b7d2,SOUOHQI12A8C14224C,5,8621,17593
2,c79500b697b2397307870c83c70b4cbad1bdf508,SOUQJDY12A6701D2EF,6,32557,17660
3,8fcd26be76c160e14e1555079b7bd0075ddbe5f8,SOVXTZF12A670207D2,5,23391,18644
4,51737e957624c4dd888ae6ea6cd066754e43553f,SODNDDS12AB017F3D1,31,13250,3074


In [ ]:
triplets.describe()

,play_count,user_id_idx,song_id_idx
count,819961.000000,819961.000000,819961.000000
mean,11.033927,20944.489382,11040.432118
std,10.289562,12101.366040,6245.041534
min,5.000000,0.000000,0.000000
25%,6.000000,10406.000000,5640.000000
50%,8.000000,20978.000000,11224.000000
75%,12.000000,31387.000000,16367.000000
max,150.000000,41905.000000,21839.000000


In [ ]:
n_users = triplets.user_id.nunique()
n_songs = triplets.song_id.nunique()

In [ ]:
output_dir = Path('../experiments/triplets')
output_dir.mkdir(parents=True, exist_ok=True)

train_triplets.to_csv(output_dir / "train.csv", index=False)
test_triplets.to_csv(output_dir / "test.csv", index=False)

In [ ]:
train_triplets

## Triplets

In [ ]:
def to_fm_sparse_fmt(rating, uid, sid):
  # order of features: user, movie, tags
    user_start_idx = 0
    song_start_idx = n_users
    
    return "{} {}:1 {}:1 \n".format(
        rating,
        uid,
        sid + song_start_idx
    ).encode('ascii')

output_dir = Path('../experiments/triplets')
output_dir.mkdir(parents=True, exist_ok=True)

meta_columns = ['play_count', 'user_id_idx', 'song_id_idx']

train_df = (
    train_triplets
    .sample(frac=1, random_state=SEED)
)
print(train_df.shape)
test_df = (
    test_triplets
    .sample(frac=1, random_state=SEED)
)
print(test_df.shape)

# save training data to file
with open(output_dir / "train.libfm", "wb") as f:
    for rating, uid, sid, *_ in train_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid))

# save test data to file
with open(output_dir / "test.libfm", "wb") as f:
    for rating, uid, sid, *_ in test_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid))
test_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"test.csv", index=False)

# save all data to file
with open(output_dir / "all.libfm", "wb") as f:
    for rating, uid, sid, *_ in triplets[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid))
triplets[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"all_df_triplets.csv", index=False)

#import random
#random.seed(SEED)
#uids = random.sample(list(triplets.user_id.unique()), 1000)
#with open(output_dir / "auc.libfm", "wb") as f:
#    for uid in uids:
#        for sid in range(n_songs):
#            f.write(to_fm_sparse_fmt(5, uid, sid+1))

(655968, 5)
(163993, 5)


## Metadata

In [ ]:
conn = crud.create_connection("../db/track_metadata.db")
tables = crud.get_tables(conn)

for table_name in tables.name.tolist():
    print(table_name)
    records = crud.get_records(conn, table_name)
    #print(records.head())

songs


In [ ]:
tracks = records[['track_id', 'title', 'song_id']]
tracks = tracks.merge(song_mapping, on='song_id')
tracks.to_csv('./tracks.csv', index=False)
tracks.shape

(22117, 4)

In [ ]:
records_columns = ['song_id', 'artist_id', 'duration', 'artist_familiarity', 'artist_hotttnesss']
songs = records[records_columns].groupby(['song_id', 'artist_id']).mean().reset_index()
songs.shape

(999056, 5)

In [ ]:
songs_lookup = (
    triplets.merge(songs, on='song_id')[['song_id', 'song_id_idx', 'duration', 'artist_familiarity', 'artist_hotttnesss']]
    .drop_duplicates()
    .set_index('song_id_idx')
)

In [ ]:
def to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness):
  # order of features: user, movie, tags
    user_start_idx = 0
    song_start_idx = n_users
    duration_start_idx = song_start_idx + n_songs
    familiarity_start_idx = duration_start_idx + 1
    hotness_start_idx = familiarity_start_idx + 1
    
    return "{} {}:1 {}:1 {}:{} {}:{} {}:{}\n".format(
        rating,
        uid,
        sid + song_start_idx, 
        duration_start_idx, duration,
        familiarity_start_idx, familiarity,
        hotness_start_idx, hotness
    ).encode('ascii')

output_dir = Path('../experiments/metadata')
output_dir.mkdir(parents=True, exist_ok=True)

meta_columns = ['play_count', 'user_id_idx', 'song_id_idx', 'duration', 'artist_familiarity', 'artist_hotttnesss']

train_df = (
    train_triplets
    .merge(songs.drop(columns=['artist_id']), on='song_id')
    .sample(frac=1, random_state=SEED)
)
print(train_df.shape)
test_df = (
    test_triplets
    .merge(songs.drop(columns=['artist_id']), on='song_id')
    .sample(frac=1, random_state=SEED)
)
print(test_df.shape)
all_df = (
    triplets
    .merge(songs.drop(columns=['artist_id']), on='song_id')
    .sample(frac=1, random_state=SEED)
)

# save training data to file
with open(output_dir / "train.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, *_ in train_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness))

# save test data to file
with open(output_dir / "test.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, *_ in test_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness))
test_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"test.csv", index=False)

with open(output_dir / "all.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, *_ in all_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness))
all_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"all_df_triplets.csv", index=False)

#import random
#random.seed(SEED)
#uids = random.sample(list(triplets.user_id.unique()), 1000)
#with open(output_dir / "auc.libfm", "wb") as f:
#    for uid in uids:
#        for sid in range(n_songs):
#            f.write(to_fm_sparse_fmt(5, uid, sid+1, songs_lookup.loc[sid].duration, songs_lookup.loc[sid].artist_familiarity, songs_lookup.loc[sid].artist_hotttnesss))

(655968, 8)
(163993, 8)


KeyboardInterrupt: 

## Artist Terms

In [ ]:
conn = crud.create_connection("../db/artist_term.db")
tables = crud.get_tables(conn)

for table_name in tables.name.tolist():
    #print(table_name)
    records = crud.get_records(conn, table_name)
    #print(records.shape)
    if table_name == "artist_term":
        break

In [ ]:
artist_term = records
artist_term['term_id'] = artist_term.groupby('term').ngroup()
n_terms = artist_term.term_id.max()
artist_term_lookup = artist_term[['artist_id', 'term_id']].drop_duplicates()
artist_term_lookup.to_csv("./artist_term_lookup.csv", index=False)
artist_term = artist_term[['artist_id', 'term_id']].groupby('artist_id').agg(list).reset_index()
artist_term.to_csv("./artist_term.csv", index=False)
artist_term.shape

(43943, 2)

In [ ]:
from pathlib import Path

def to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, terms):
  # order of features: user, movie, tags
    user_start_idx = 0
    song_start_idx = n_users
    duration_start_idx = song_start_idx + n_songs
    familiarity_start_idx = duration_start_idx + 1
    hotness_start_idx = familiarity_start_idx + 1
    artist_term_start_idx = hotness_start_idx + 1
    
    return "{} {}:1 {}:1 {}:{} {}:{} {}:{} {}\n".format(
        rating,
        uid,
        sid + song_start_idx,
        duration_start_idx, duration,
        familiarity_start_idx, familiarity,
        hotness_start_idx, hotness,
        " ".join("{}:1".format(t + artist_term_start_idx) for t in terms)
    ).encode('ascii')

output_dir = Path('../experiments/metadata_artist')
output_dir.mkdir(parents=True, exist_ok=True)

meta_columns = ['play_count', 'user_id_idx', 'song_id_idx', 'duration', 'artist_familiarity', 'artist_hotttnesss', 'term_id']

train_df = (
    train_triplets
    .merge(songs, on='song_id')
    .merge(artist_term, on="artist_id")
    .drop(columns=['artist_id'])
)
print(train_df.shape)
test_df = (
    test_triplets
    .merge(songs, on='song_id')
    .merge(artist_term, on="artist_id")
    .drop(columns=['artist_id'])
)
print(test_df.shape)
all_df = (
    triplets
    .merge(songs, on='song_id')
    .merge(artist_term, on="artist_id")
    .drop(columns=['artist_id'])
)

# save training data to file
with open(output_dir / "train.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, terms, *_ in train_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, terms))

# save test data to file
with open(output_dir / "test.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, terms, *_ in test_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, terms))
test_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"test.csv", index=False)

with open(output_dir / "all.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, terms, *_ in all_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, terms))
all_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"all_df_triplets.csv", index=False)

(655684, 9)
(163931, 9)


## Lyrics

In [ ]:
conn = crud.create_connection("../db/mxm_dataset.db")
tables = crud.get_tables(conn)

for table_name in tables.name.tolist():
    #print(table_name)
    records = crud.get_records(conn, table_name)
    #print(records.shape)

In [ ]:
lyrics = records 
lyrics['word_id'] = lyrics.groupby('word').ngroup()
n_lyrics = lyrics.word_id.max()
lyrics_word_lookup = lyrics[['word_id', 'word']].drop_duplicates()
lyrics_word_lookup.to_csv("./lyrics_word_lookup.csv", index=False)
lyrics = lyrics.merge(tracks[['track_id', 'song_id']], on='track_id')[['song_id', 'word_id', 'count']].drop_duplicates()
lyrics = lyrics.groupby('song_id')[['word_id', 'count']].apply(lambda g: g.values.tolist()).reset_index()
lyrics.columns = ['song_id', 'lyrics']
lyrics.to_csv("./lyrics.csv", index=False)
lyrics.shape

(13396, 2)

In [ ]:
from pathlib import Path

def to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, lyric):
  # order of features: user, movie, tags
    user_start_idx = 0
    song_start_idx = n_users
    duration_start_idx = song_start_idx + n_songs
    familiarity_start_idx = duration_start_idx + 1
    hotness_start_idx = familiarity_start_idx + 1
    lyric_start_idx = hotness_start_idx + 1
    
    return "{} {}:1 {}:1 {}:{} {}:{} {}:{} {}\n".format(
        rating,
        uid,
        sid + song_start_idx,
        duration_start_idx, duration,
        familiarity_start_idx, familiarity,
        hotness_start_idx, hotness,
        " ".join("{}:{}".format(t[0] + lyric_start_idx, t[1]) for t in lyric)
    ).encode('ascii')

output_dir = Path('../experiments/metadata_lyrics')
output_dir.mkdir(parents=True, exist_ok=True)

meta_columns = ['play_count', 'user_id_idx', 'song_id_idx', 'duration', 'artist_familiarity', 'artist_hotttnesss', 'lyrics']

train_df = (
    train_triplets
    .merge(songs, on='song_id')
    .merge(lyrics, on="song_id")
    .drop(columns=['artist_id'])
)
print(train_df.shape)
test_df = (
    test_triplets
    .merge(songs, on='song_id')
    .merge(lyrics, on="song_id")
    .drop(columns=['artist_id'])
)
print(test_df.shape)
all_df = (
    triplets
    .merge(songs, on='song_id')
    .merge(lyrics, on="song_id")
    .drop(columns=['artist_id'])
)

# save training data to file
with open(output_dir / "train.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, lyric, *_ in train_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, lyric))

# save test data to file
with open(output_dir / "test.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, lyric, *_ in test_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, lyric))
test_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"test.csv", index=False)

with open(output_dir / "all.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, lyric, *_ in all_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, lyric))
all_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"all_df_triplets.csv", index=False)

(398494, 9)
(99838, 9)


## Spotify

In [ ]:
spotify_id = pd.read_csv('./spotify_ids.csv', header=None, sep='|', error_bad_lines=False, warn_bad_lines=False, names=
                  ['query', 'spotify_title', 'spotify_artist', 'spotify_album', 'spotify_release', 'spotify_uri', 'track_id']
                  )
spotify_id = spotify_id.loc[~spotify_id.spotify_uri.isna()]
spotify_id = spotify_id.merge(tracks[['track_id', 'song_id']], on='track_id')[['spotify_uri', 'song_id', 'spotify_release']].drop_duplicates()
spotify_id.shape

(17827, 3)

In [ ]:
audio_features = pd.read_csv("./spotify_audio_features.csv")
audio_features = audio_features.add_prefix('spotify_')
print(audio_features.shape)
audio_features = audio_features.loc[audio_features.spotify_energy!=-1]
print(audio_features.shape)
audio_features.shape

(790704, 13)
(790599, 13)


(790599, 13)

In [ ]:
spotify = spotify_id.merge(audio_features, on="spotify_uri").drop(columns=['spotify_uri', 'spotify_release']).drop_duplicates()
spotify = spotify.groupby('song_id').mean().reset_index()
spotify.to_csv("./spotify.csv", index=False)

In [ ]:
spotify = spotify_id.merge(audio_features, on="spotify_uri").drop(columns=['spotify_uri', 'spotify_release']).drop_duplicates()
spotify = spotify.groupby('song_id').mean().reset_index()
print(spotify.song_id.nunique())
print(spotify.shape)
spotify_cols = [col for col in spotify.columns if col != 'song_id']
spotify = pd.melt(spotify, id_vars=['song_id'], value_vars=spotify_cols)
spotify = spotify.groupby('song_id')[['variable', 'value']].apply(lambda g: g.values.tolist()).reset_index()
spotify.columns = ['song_id', 'spotify']
spotify.shape

17818
(17818, 13)


(17818, 2)

In [ ]:
def to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, spotify_audio):
  # order of features: user, movie, tags
    user_start_idx = 0
    song_start_idx = n_users
    duration_start_idx = song_start_idx + n_songs
    familiarity_start_idx = duration_start_idx + 1
    hotness_start_idx = familiarity_start_idx + 1
    audio_start_idx = hotness_start_idx + 1
    
    return "{} {}:1 {}:1 {}:{} {}:{} {}:{} {}\n".format(
        rating,
        uid,
        sid + song_start_idx,
        duration_start_idx, duration,
        familiarity_start_idx, familiarity,
        hotness_start_idx, hotness,
        " ".join("{}:{}".format(spotify_cols.index(s[0]) + audio_start_idx, s[1]) for s in spotify_audio)
    ).encode('ascii')

output_dir = Path('../experiments/metadata_spotify')
output_dir.mkdir(parents=True, exist_ok=True)

meta_columns = ['play_count', 'user_id_idx', 'song_id_idx', 
                'duration', 'artist_familiarity', 'artist_hotttnesss', 
                'spotify']

train_df = (
    train_triplets
    .merge(songs, on='song_id')
    .merge(spotify, on="song_id")
    .drop(columns=['artist_id'])
    .sample(frac=1, random_state=SEED)
)
print(train_df.shape)
test_df = (
    test_triplets
    .merge(songs, on='song_id')
    .merge(spotify, on="song_id")
    .drop(columns=['artist_id'])
    .sample(frac=1, random_state=SEED)
)
print(test_df.shape)
all_df = (
    triplets
    .merge(songs, on='song_id')
    .merge(spotify, on="song_id")
)
print(all_df.shape)

# save training data to file
with open(output_dir / "train.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, spotify_audio, *_ in train_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, spotify_audio))

# save test data to file
with open(output_dir / "test.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, spotify_audio, *_ in test_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, spotify_audio))
test_df[['user_id', 'song_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"test.csv", index=False)

with open(output_dir / "all.libfm", "wb") as f:
    for rating, uid, sid, duration, familiarity, hotness, spotify_audio, *_ in all_df[meta_columns].itertuples(index=False):
        f.write(to_fm_sparse_fmt(rating, uid, sid, duration, familiarity, hotness, spotify_audio))
        
all_df[['user_id', 'song_id', 'artist_id', 'play_count', 'user_id_idx', 'song_id_idx']].to_csv(output_dir/"all_df_spotify.csv", index=False)

(537777, 9)
(134040, 9)
(671817, 10)
